Neural networks can sometimes be easy fooled. Try to create a random looking image that makes the NN from [CNNIntro.ipynb](CNNIntro.ipynb) think that it sees a cat. You need to run that notebook before to get the model!

In [ ]:
import keras
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
model = keras.models.load_model("CNNIntro_model.h5")

Start with this random image:

In [ ]:
rnd_img = 0.1*np.random.rand(1, 32, 32, 3)

In [ ]:
plt.imshow(rnd_img[0])

What does our model say to this?

In [ ]:
plt.bar(labels, model.predict(rnd_img)[0])
plt.xticks(rotation=90)

Let's see if we can modify the image such that the NN thinks this is a cat. So we want to maximise the output for cat:

In [ ]:
make_this_a = "cat"
cat_loss = K.mean(model.output[:,labels.index(make_this_a)])
# cat_loss is a Tensor:
print(cat_loss)

By calculating the gradient w.r.t the input image

In [ ]:
grad_inp_fn = K.function([model.input], K.gradients(cat_loss, [model.input]))

Now, call this function on the input image and create a new one by successively following the gradient for a few steps and check if the neural network then sees a cat. The function is called with a list of one element in this case and returns one element:

In [ ]:
grad_inp_fn([rnd_img])

In [ ]:
#model.predict(rnd_img)[0]
#cat_loss = K.mean(model.output[:,labels.index('cat')])
#grad_inp_fn = K.function([model.input], K.gradients(cat_loss, [model.input]))
for it in range(1000):
    grad = grad_inp_fn([rnd_img])
    rnd_img += 0.1*grad[0]
    model.predict(rnd_img)[0]
    prob_cat = model.predict(rnd_img)[0][labels.index(make_this_a)]
    if it < 10 or (it % 5) == 0:
        print("%d: %.3f %s" % (it, prob_cat, make_this_a))
    if prob_cat > 0.995:
        break
plt.bar(labels, model.predict(rnd_img)[0])
plt.xticks(rotation=90)

In [ ]:
plt.imshow(rnd_img[0]*5)

Now we just do the same for all 10 labels that are used for the classification of the image set and compare the different results.

In [ ]:
imgs = {}
for label in labels:
    print("Training for %s" % label)
    loss = K.mean(model.output[:,labels.index(label)])
    grad_inp_fn = K.function([model.input], K.gradients(loss, [model.input]))
    rnd_img = 0.1*np.random.rand(1, 32, 32, 3)
    for it in range(1000):
        grad = grad_inp_fn([rnd_img])
        rnd_img += 0.1*grad[0]
        model.predict(rnd_img)[0]
        prob = model.predict(rnd_img)[0][labels.index(label)]
        if prob > 0.995:
            break
    imgs[label] = rnd_img

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(20,6))
idx = 0
for label, img in imgs.items():
    axs.reshape(-1)[idx].imshow(img[0]*10)
    axs.reshape(-1)[idx].set(xlabel=label)
    idx += 1
